# Transfer learning with Tensorflow Part 1: Feature Extraction

Transfer learning is leveraging a working model's exixting architecture and learned paterns for our own problem.

Why use transfer learning

### Change runtime type to GPU


In [ ]:
!nvidia-smi


## Download and becoming one with the data

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

## Create some data loaders (preparing the data)

we'll use the ImageDataGenerator class ti load in our images in batches.

In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMG_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMG_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

## Setting up callbacks (things to run whilst our model is training)

Callbacks are extra functionalities to add to our model during or after training.

Porpolar callbacks:
* Tracking experiements with TensorBoard callback
* Model checkpoint with ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with Earlystopping callback 

In [ ]:
# Create a TensorBoard callback
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving Tensorflow log files to: {log_dir}")
    return tensorboard_callback

## Creating models using TensorFlow Hub

In the past we've used Tensorflow to create my models layer by layer from scratch.

Now  majority of our models layers is going to come from TensorFlow Hub.

We can access pretrained models on: https://tfhub.dev/

Browsing the TensorFlow Hub page and sorthing for image classification, we found the following feature vector model link: https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

In [ ]:
# Let's compare the following two models 
resnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
# Import dependecies
import tensorflow as tf 
import tensorflow_hub as hub
from tensorflow.keras import layers 

In [ ]:
# Let's make a create_model() function to create a model from url
def create_model(model_url, num_classes=10):
  """
  Take a Tensorflow Hub URL and create a keras Sequential model with it.

  Agrs:
    model_url (str): A Tensorflow Hub feature extraxction URL.
    num_classes (int): Number of output neurons in the output layer, 
    should be equal to number of target classes, default 10.

  Returns an uncompiled kera Sequential model with model_url as feature extractor layer
  and dense output layer with num_classes output neurons.
  """
  # Download the pretrained model and save it as a keras layer 
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the already learned patterns
                                           name="feature_extraction_layer",
                                           input_shape=IMG_SHAPE+(3,)) 
  
  # Create our own model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])

  return model

### Creating and testing ResNet Tensorflow Hub Feature Extraction model

In [ ]:
# Create ResNet model
resnet_model =  create_model(resnet_url,
                             num_classes=train_data_10_percent.num_classes)

In [ ]:
# Compile our resnet model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["Accuracy"])

In [ ]:
# Let's fit our ResNet model to the data (10 percent of 10 classes)
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50v2"
                                                                         )])

In [ ]:
resnet_model.summary()

In [ ]:
 # Let's create a function to plot our loss curves...